In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
# pd.set_option('display.max_rows',None)

적당한 머지 이후 
> 유저별로 충성도를 확인 (시간 / 횟수 / 구매) 해볼 필요가 있음

# 1) history_data.csv : 시청 시작 데이터
![image](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000004/files/2b762359-78b9-470c-9369-5d1b3d9ff06f..png)

profile_id 8,311명의 유저가 100만건의 시청기록을 만들었다.    

act_target_dtl 의 모든 값은 -003으로 통일.   

album_id  20,695개의 동영상이 100만건 가량의 시청기록을 만들었다.    

continuous_play는 9:1 정도의 class imbalance를 보인다.     

short_trailer는 추천에 큰 영향을 미치지 않을 것으로 에상된다.     

payment 금액 분포가 수상하다. 단지 동영상 구매뿐만이 아니라 체험학습 구매나 예약 등도 같이 이뤄진 게 아닌가?   

> 시청 종료 데이터와 merge할 수 있을까? 

### 2022년 3월 1일부터~ 2022년 4월 27일까지의 데이터

In [ ]:
history = pd.read_csv('../input/lgground/history_data.csv')
history['payment'].fillna(0,inplace=True)

In [ ]:
history

In [ ]:
history.info()

In [ ]:
# 세션 ID 개수와 동영상 ID 개수가 일치한가?  ==  중복시청기록이 남아 있는가?
history.groupby(['profile_id']).count()

# a = history.groupby(['profile_id'])['album_id'].count().sort_values()
# a.tail(20)
# b = pd.DataFrame(history.groupby(['profile_id'])['ss_id'].count().sort_values())
# b.tail(20)

>> 중복 시청 기록은 데이터에 없다. <<
세션 수 == 동영상 ID 수.     
숫자 차이가 났다면,(세션 수 > 동영상 ID 였다면 중복시청기록이 데이터에 남아 있다는 뜻)

In [ ]:
# 최다시청자를 찾아보자 / 시청분포확인
# id 3847,2794는 무서울 정도로 많이 봐서 우선 제외 (14014, 18190편씩 봄)
# pd.DataFrame(history.groupby(['profile_id'])['album_id'].count().sort_values())

a = pd.DataFrame(history.groupby(['profile_id'])['album_id'].count().sort_values().drop([3847,2794]))
a

### submission file의 row 수  : 8311명

In [ ]:
num_hispro = history['profile_id'].nunique()
num_albpro = history['album_id'].nunique()
num_paypro = history['payment'].count()
print(f"시작 프로필 수 : {num_hispro}")
print(f"시작 아이템 수 : {num_albpro}")
print(f"구매 수 : {num_paypro}")

In [ ]:
pd.DataFrame(history['payment'].value_counts())
# 금액분포 무슨일임?
# 실제 동영상 가격일지 확인하는 작업이 필요할지도 모르겠다. 또한 Nan도 다수 있다. 
# 가격이 0인 동영상이 없다. > 0원 == NaN  위에서 결측치 채워버리고 해결.
# 가격 높은 동영상 =?= 부모용 컨텐츠?

In [ ]:
pie_data = history['continuous_play'].value_counts()
print(pie_data)

sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
colors = sns.color_palette('pastel')
ax.pie(pie_data, labels=pie_data.index, colors = colors)
ax.set_facecolor('lightgrey')
ax.set_xlabel('continuous_play')
plt.show()

# 거의 모든 시청이 '연속 시청'으로 이루어진다.

# 2) watch_e_data.csv : 시청 종료 데이터
![watch](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000004/files/81a5741f-9481-49ed-8533-1e62d5f9fb50..png)

시청 시작 데이터와 시청 종료 데이터 사이의 row 개수 차이가 심각함. 112,857개 차이가 난다.   

profile_id 에서도 nunique가 시작 데이터보다 종료 데이터가 635명 적다.   

album_id 도   nunique (20695 - 20361) 334개 더 적다.   

watch_time 은 적절한 구간별 전처리가 필요함. 실수값이 심하다. 게다가 빈도로 20개를 확인했을 때, 시청시간 0초~7초가 너무 많다. 이걸 다 neg sampling으로 하기에도 애매하다. 영상 최대 4769초(79분짜리) 시청.    
> 잘못 틀었다가 다른 것으로 넘긴 것인지 확인이 필요 (continuous play와 연계한 파악이 가능한지?) 

total_time 의 최대 영상 길이는 8929초 (149분). 평균적으로 271초(4.5분). Box plot으로 분포나 아웃라이어 확인해볼 필요가 있다.   

continuous_play 는 1(연속재생)이 과반수인 5만건을 차지하며, 0(이후 재생 없음)은 3만건을 차지한다. 이 column은 무의미하다고 보여진다. **중요한 건, 절반 이상의 유저가 연속재생으로 컨텐츠를 소비하고 있다는 점**이다.    

In [ ]:
watch = pd.read_csv('../input/lgground/watch_e_data.csv')
watch

In [ ]:
watch.info()
# 시청 시작 데이터와 row 갯수 차이가 심각함. 112,857개 차이가 난다. (merge 불가능하지 않아?)

In [ ]:
watpro = watch['profile_id'].nunique()
print(f"종료 프로필 수 : {watpro}")
watalb = watch['album_id'].nunique()
print(f"종료 아이템 수 : {watalb}")

# 시작 프로필 수 : 8311  (653개 차이)
# 시작아이템 수 : 20695 (334개 차이)

In [ ]:
watch['watch_time'].describe()

# 시청 시간 기초통계량 확인 (초 단위)
# 최대 시청 시간 : 4769 초

In [ ]:
watch_df = pd.DataFrame(watch['watch_time'].reset_index(drop=True))
# watch_df = pd.DataFrame(watch_df, columns='watch_time')
watch_df

# 시청시간 전체의 분포를 확인하기 위한 작업 

In [ ]:
sns.set_theme(style='whitegrid')
 
ax = sns.displot(data=watch_df, x='watch_time', binwidth=3,stat="count", discrete=True)

In [ ]:
pd.DataFrame(watch.groupby(['profile_id']).count()['watch_time'].sort_values())

# 유저당 시청 횟수 (프로필 id 별로 시청횟수 집계)
# 최다 시청횟수 13189 . 최소 시청횟수 1

In [ ]:
# cold start 유저 목록 뽑기

cold_watcher = pd.DataFrame(watch.groupby(['profile_id']).count()['watch_time'].sort_values())
cold_watcher.describe()

In [ ]:
cold_watcher_34 = cold_watcher[(cold_watcher['watch_time']<34)].reset_index()
cold_watcher_34
# 348명의 cold start 유저 존재 (threshold 1)
# 하위 25% 미만은 cold 하다고 가정? << 이 가정이 맞을지는 추후 확인  ( 이 경우, threshold 33. 3712명이 cold 하다는 판단이 됨)

######## cold start 유저 리스트 따로 뽑아두기  
df_dict = cold_watcher_34.to_dict()
dic_cold = df_dict['profile_id'].values()
coldlist = list(dic_cold)
# coldlist

# 특히 영상 총 시간이 긴 item의 장르, 제목이 무엇인지 확인해 볼 필요 있음 <아래 참조>  

In [ ]:
watch['total_time'].value_counts()[:30]

# 동영상 최대 길이가 얼마나 되는지 가볍게 확인

In [ ]:
watch['total_time'].describe()

## 최대 시간이 149분이면 이건 100% 부모용 영상이다 

In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
ax = sns.boxplot(data=watch, x='total_time', color='purple')
ax.set_xlabel('total outliers')
plt.show()

In [ ]:
watch['continuous_play'].value_counts()

In [ ]:
play_df = pd.DataFrame(watch['continuous_play'].value_counts().reset_index()) 

In [ ]:
play_df = pd.DataFrame(watch['continuous_play'].value_counts().reset_index()) 
type=['이후 컨텐츠 재생 없음','연속재생','선택에 의한 종료','기타 종료']

# 3) buy_data.csv : 구매 이력 데이터

![column](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000004/files/84d3a789-1a48-46fa-87df-3fcd42efd28b..png)


profile_id의 nquniqe는 전체 17명밖에 되지 않는다. 150건의 구매를 17명이 냈다는 뜻.   
   
ss_id와 log_time 2개 column은 시간적으로 큰 차이가 안 난다. 특별한 의미가 없어 보인다. 두 column중 하나만 남겨서, merge혹은 train-test split에 사용하면 될 듯. *다만 시청 log_time의 경우에는 새로운 추천BM 제안(시간대별 추천 등)에는 쓸모가 있을지도 모른다.*    
    ㄴ참조 : 새로운 추천 아이디어 및 프로필별 맞춤형 콘텐츠 추천 AI 모델을 개발하는 AI 경진대회입니다.   
   
MKID004은 unique한 값이 1개인 column하다. Drop 해도 될 듯.
   
Payment column은 가격이 구매영상 선택과 추천에 있어 중요한 요소가 될 수도 있다는 점을 시사하고, 가격이 특히 높은 것이 무엇이었는지 확인해 볼 필요도 있어 보인다.    
300~500원대 영상이 가장 많이 구매되었다.    
> 확인해볼것+ 구매하지 않아도 볼 수 있는 영상/구매해야만 볼 수 있는 영상이 나뉘어 있을지도 모름    

In [ ]:
buy = pd.read_csv('../input/lgground/buy_data.csv')
buy.sort_values('ss_id')

In [ ]:
buy.info()

In [ ]:
buypro = buy['profile_id'].nunique()
print(f"구매 유저 명수 : {buypro}명")
buyalb = buy['album_id'].nunique()
print(f"구매된 영상 개수 : {buyalb}개")

In [ ]:
# 구매된 영상의 이름, 장르, 카테고리 확인   ##### Meta 데이터셋과 머지
buy_df = pd.merge(buy, meta, on = 'album_id', how = 'left').drop(labels=['ss_id','log_time','genre_mid','genre_small','onair_date','cast_2','cast_3','cast_4','cast_5','cast_6','cast_7'],axis=1)

In [ ]:
# 판매 금액대별로 묶기 
buy_popular = buy_df.groupby(['payment','album_id']).count().drop(labels=['act_target_dtl','title','sub_title','genre_large','country','run_time','cast_1'],axis=1)
buy_popular

In [ ]:
# 판매 금액대별로 묶은 것들 확인했으니, 이제 dict 형식으로 저장해두기.   <album ID - 판매 횟수 >
buy_alid = buy_popular.reset_index().drop(labels='payment',axis=1).rename(columns={'profile_id':'buy_count'})
buy_alid_dict =  buy_alid.to_dict('list')

# buy_df (메인 머지) 데이터프레임에 구매 횟수 column 추가해두기 <추후 meta 데이터셋에 추가되어야 하는 내용> 
buy_count = pd.DataFrame.from_dict(buy_alid_dict)
buy_df = pd.merge(buy_df, buy_count, on = 'album_id', how = 'left')
buy_df

In [ ]:
buy_df['payment'].unique()

In [ ]:
buy_df[(buy_df['payment']==6300)]

In [ ]:
buy_df[(buy_df['payment']==4200)]

In [ ]:
buy_df[(buy_df['payment']==2500)]

In [ ]:
buy_df[(buy_df['payment']==1500)]

In [ ]:
# 가장 많이 팔린 횟수는? 
buy_df['buy_count'].value_counts()

# 1회 구매가 압도적으로 많음 

In [ ]:
# 판매에 기여한 캐릭터 순위 확인 (인기 캐릭터 순위)

################ 뽀로로 > 핑크퐁 > 콩순이 > 렉스 > 폴리 

# 나이별 인기 캐릭터 순위도 확인 필요 

buy_df.groupby(['cast_1']).count()

In [ ]:
buy['payment'].describe()

In [ ]:
pd.DataFrame(buy['payment'].value_counts())

In [ ]:
# 최다 구매 유저 <충성고객>
buy_df.groupby(['profile_id']).count().sort_values('act_target_dtl',ascending=False)[:11]

# 4) search_data.csv : 검색을 통한 시청 데이터
![search](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000004/files/4fdc2a58-e698-4bbc-b2a7-ffa20ccb6a61..png)

In [ ]:
search = pd.read_csv('../input/lgground/search_data.csv')

In [ ]:
ser_df = pd.merge(search, meta, on = 'album_id', how = 'left').drop(labels=['ss_id','log_time','genre_mid','genre_small','onair_date','cast_2','cast_3','cast_4','cast_5','cast_6','cast_7'],axis=1)
ser_df

In [ ]:
seapro = search['profile_id'].nunique()
print(f"검색 시도한 유저 명수 : {seapro}")
seaalb = search['album_id'].nunique()
print(f"검색된 아이템 개수 : {seaalb}")

In [ ]:
# 최다검색유저
ser_df.groupby(['profile_id']).count().sort_values('act_target_dtl',ascending=False)[:30]

In [ ]:
# 가장 많이 검색된 동영상 top 30

ser_df.groupby(['album_id']).count().sort_values('profile_id',ascending=False)[:30]

In [ ]:
# 자주 검색된 장르 확인

ser_df['genre_large'].value_counts()

In [ ]:
# 많이 검색된 title 순위
pd.DataFrame(ser_df['title'].value_counts()[:20])

In [ ]:
pd.DataFrame(ser_df['sub_title'].value_counts()[:20])

# 검색도 많이 되고 구매도 많이 된 영상 목록 확인 필요

# 5) meta_data.csv : 콘텐츠 일반 메타 정보
![meta](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000004/files/945112b2-71b4-43a5-bb57-ef4b49fe6bff..png)

In [ ]:
meta = pd.read_csv('../input/lgground/meta_data.csv')
meta

In [ ]:
meta.info()

In [ ]:
meta['album_id'].nunique()  #
print()

In [ ]:
meta['title'].nunique()

# 이거 album_id가 3천개 가량 더 많다? 한 영상에 여러 개의 id가 할당되어 있는듯

In [ ]:
meta['cast_1'].value_counts()  #역시 뽀통령

# 6) meta_data_plus.csv : 콘텐츠 확장 정보
![meplus](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00001436/files/d1c0ed82-8e75-440d-91e8-e765c8f31d89..png)

In [ ]:
metaplus = pd.read_csv('../input/lgground/meta_data_plus.csv')
metaplus

In [ ]:
metaplus['keyword_type'].nunique()

In [ ]:
metaplus['keyword_name'].nunique()

# 태그코드 고유값이 태크코드명보다 40개 가량 많다. 

# 7) profile_data.csv : 프로필 정보
![profile](https://s3-us-west-2.amazonaws.com/aistages-prod-server-public/app/Users/00000004/files/a3c750f4-b97d-488f-aaea-ef0e334b86e5..png)

profile_id 8311개    

sex 성비는 여아가 600명 가량 더 많았다.   

age  최소나이 1살, 최대나이 13살. 3~8살 사이가 70% 가량을 차지한다.   

pr_interest_keyword_cd_1 부모의 선호 키워드 순위는 6>2>>>1>7>3   

ch_interest_keyword_cd_1 아이의 선호 키워드 순위는 1>>>>>>2>3>>46


In [ ]:
profile = pd.read_csv('../input/lgground/profile_data.csv')
profile

In [ ]:
profile.info()

In [ ]:
profile['age'].describe()

In [ ]:
pie_data = profile[['profile_id', 'age']].groupby('age').count()

In [ ]:
sns.set_style("darkgrid")
f, ax = plt.subplots(figsize=(10,5))
colors = sns.color_palette('pastel')
ax.pie(pie_data.profile_id, labels=pie_data.index, colors = colors)
ax.set_facecolor('lightgrey')
ax.set_xlabel('age')
plt.show()

In [ ]:
profile['sex'].value_counts()

In [ ]:
f, ax = plt.subplots(figsize=(15, 7))
ax = sns.histplot(data=profile, y='pr_interest_keyword_cd_1', color='orange')
ax.set_xlabel('count by keyword name')
ax.set_ylabel('index name')
plt.show()

In [ ]:
profile['pr_interest_keyword_cd_1'].value_counts()

In [ ]:
f, ax = plt.subplots(figsize=(15, 7))
ax = sns.histplot(data=profile, y='ch_interest_keyword_cd_1', color='orange')
ax.set_xlabel('count by keyword name')
ax.set_ylabel('index name')
plt.show()

In [ ]:
profile['ch_interest_keyword_cd_1'].value_counts()

In [ ]:
# 나이별 선호 키워드 확인해보기 (아이들)
pd.set_option('display.max_rows',None)

In [ ]:
pd.DataFrame(profile.groupby(['age','ch_interest_keyword_cd_1']).count()['profile_id']) #이쁘게 만든다 